In [7]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [19]:
import pandas as pd
import numpy as np
sentence = 'Thomas Jefferson geban building Monticello at the age of 26.'
token_setence = sentence.split()
vocab = sorted(set(token_setence))
onehot = np.zeros((len(token_setence),len(vocab)),int)
for i,word in enumerate(token_setence):
    onehot[i,vocab.index(word)] = 1
# print(onehot)
onehot = pd.DataFrame(onehot,columns=vocab)
# print(onehot)
onehot[onehot==0] = '    '
print(onehot)

    26. Jefferson Monticello Thomas   age    at building geban    of   the
0                                 1                                       
1               1                                                         
2                                                            1            
3                                                      1                  
4                          1                                              
5                                             1                           
6                                                                        1
7                                       1                                 
8                                                                  1      
9     1                                                                   


In [40]:
#构建词袋向量
sentence = 'Thomas Jefferson geban building Monticello at the age of 26.\n'
sentence += 'Jefferson inherited sizable property in Albemarle County, Virginia, from his father, Peter Jefferson.\n'
sentence += ' Jefferson began planning revisions for his Albemarle County home, based in part on what he had observed in France.\n'
sentence += 'To achieve this effect, windows in the second-story bedrooms are on the floor lever.\n'
corpus = {}
# print(sentence.split('\n'))
for i,sent in enumerate(sentence.split('\n')):
    corpus['sent{0}'.format(i)] = dict((tok,1) for tok in sent.split())
#     print(corpus['sent{0}'.format(i)])
#     break
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df.iloc[:,:10]
df = df.T
df.sent0.dot(df.sent1)   #第一句和第二句有个重复的词
df.sent0.dot(df.sent4)   #第一句和第五句每个单词都不一样
[ (i,j) for (i,j) in(df.sent0 & df.sent1).items() if j]    #找出相同的词


,Thomas,Jefferson,geban,building,Monticello,at,the,age,of,26.
sent0,1,1,1,1,1,1,1,1,1,1
sent1,0,1,0,0,0,0,0,0,0,0
sent2,0,1,0,0,0,0,0,0,0,0
sent3,0,0,0,0,0,0,1,0,0,0
sent4,0,0,0,0,0,0,0,0,0,0


1

0

[('Jefferson', 1)]

In [51]:
#对英文句子进行正则表达式切分
import re 
sentence = 'Thomas Jeffffffffffffferson-geban building Monticello at the age of 26.'
tokens = re.split(r'[-\s,.;!?]',sentence)
tokens

from nltk.tokenize import TreebankWordTokenizer    #内置多种英语常见的分词规则
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence)

#社交网络的非规范文本进行分词
from nltk.tokenize.casual import casual_tokenize
casual_tokenize(sentence)
casual_tokenize(sentence,reduce_len=True,strip_handles=True)

['Thomas',
 'Jeffffffffffffferson',
 'geban',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '']

['Thomas',
 'Jeffffffffffffferson-geban',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

['Thomas',
 'Jeffffffffffffferson-geban',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

['Thomas',
 'Jeffferson-geban',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

In [7]:
# 对句子实现 n-gram 分词
from nltk.util import ngrams
import re

sentence = 'Thomas Jefferson geban building Monticello at the age of 26.'
pattern = re.compile(r'[-\s.,;?!]+')
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in '-\t\n.,;!?']
tokens     # 1-gram

gram_2 = list(ngrams(tokens,2))
gram_3 = list(ngrams(tokens,3))

# ' '.join(('Thomas','Jefferson'))
gram_2 = [' '.join(x) for x in gram_2]
gram_2

['Thomas',
 'Jefferson',
 'geban',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

['Thomas Jefferson',
 'Jefferson geban',
 'geban building',
 'building Monticello',
 'Monticello at',
 'at the',
 'the age',
 'age of',
 'of 26']

In [9]:
# 停用词     同时sklearn里面也有停用词表，且更大
import nltk
import ssl
# nltk.download('stopwords')     #下载不了，手动添加的  

stop_words = nltk.corpus.stopwords.words('english')
len(stop_words)
stop_words[:7]
tokens = [i for i in tokens if i not in stop_words]
tokens   #停用之后的结果


179

['i', 'me', 'my', 'myself', 'we', 'our', 'ours']

['Thomas', 'Jefferson', 'geban', 'building', 'Monticello', 'age', '26']

In [13]:
#词汇表归一化
#大小写转换
tokens = ['House','Visitor','Center']
tokens_min = [i.lower() for i in tokens]
tokens_min

#词干还原(单复数、时态等)
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
sentence = 'dish washer\'s washed dishes'
' '.join([stemmer.stem(w).strip('\'') for w in sentence.split()])

#词性归并(找到相同意思的词)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('better',pos='a')
lemmatizer.lemmatize('best',pos='a')
lemmatizer.lemmatize('good',pos='a')     #pos = 'a'代表归并为形容词，从结果的出better和good都会归并为good,而best并不会

['house', 'visitor', 'center']

'dish washer wash dish'

'good'

'best'

'good'

#  情感分析（基于规则算法，基于机器学习）


In [8]:
#基于规则算法
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sa = SentimentIntensityAnalyzer()
len(sa.lexicon)   #共7506个词条
sentence1 = 'Python is a good tool when I use it to do something!'
sa.polarity_scores(text=sentence1)    

sentence2 = 'Horrible! Completelt useless.'
sa.polarity_scores(text=sentence2)    


7506

{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'compound': 0.4926}

{'neg': 0.868, 'neu': 0.132, 'pos': 0.0, 'compound': -0.7644}

In [2]:
#基于机器学习
from nlpia.data.loaders import get_data
movies = get_data('hutto_movies')
movies.head().round(2)
movies.describe().round(2)

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Users\\24132\\AppData\\Roaming\\Python\\Python38\\site-packages\\nlpia\\data\\hutto_ICWSM_2014/movieReviewSnippets_GroundTruth.csv.gz',), **{'nrows': None, 'low_memory': False})`...


,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [3]:
import pandas as pd
from nltk.tokenize import casual_tokenize
from collections import Counter

pd.set_option('display.width',75)
bags_of_words = []
q = 0
for text in movies.text:
    bags_of_words.append(Counter(casual_tokenize(text)))
    if q<=3:
        print(Counter(casual_tokenize(text)))
    q += 1
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape

Counter({"'": 4, 'to': 2, 'The': 1, 'Rock': 1, 'is': 1, 'destined': 1, 'be': 1, 'the': 1, '21st': 1, "Century's": 1, 'new': 1, 'Conan': 1, 'and': 1, 'that': 1, "he's": 1, 'going': 1, 'make': 1, 'a': 1, 'splash': 1, 'even': 1, 'greater': 1, 'than': 1, 'Arnold': 1, 'Schwarzenegger': 1, ',': 1, 'Jean': 1, 'Claud': 1, 'Van': 1, 'Damme': 1, 'or': 1, 'Steven': 1, 'Segal': 1, '.': 1})
Counter({'of': 4, "'": 4, '.': 4, 'The': 2, 'R': 2, 'gorgeously': 1, 'elaborate': 1, 'continuation': 1, 'Lord': 1, 'the': 1, 'Rings': 1, 'trilogy': 1, 'is': 1, 'so': 1, 'huge': 1, 'that': 1, 'a': 1, 'column': 1, 'words': 1, 'cannot': 1, 'adequately': 1, 'describe': 1, 'co': 1, 'writer': 1, '/': 1, 'director': 1, 'Peter': 1, "Jackson's": 1, 'expanded': 1, 'vision': 1, 'J': 1, "Tolkien's": 1, 'Middle': 1, 'earth': 1})
Counter({'Effective': 1, 'but': 1, 'too': 1, 'tepid': 1, 'biopic': 1})
Counter({'to': 4, 'If': 1, 'you': 1, 'sometimes': 1, 'like': 1, 'go': 1, 'the': 1, 'movies': 1, 'have': 1, 'fun': 1, ',': 1, 'Wa

(10605, 20756)

In [23]:
df_bows.head()
df_bows.head()[list(bags_of_words[0].keys())]
# list(bags_of_words[0].keys())

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Schwarzenegger,",",Jean,Claud,Van,Damme,or,Steven,Segal,.
0,1,1,1,1,2,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [36]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(df_bows,movies.sentiment>0)
movies['predicted_sentiment'] = nb.predict_proba(df_bows)[:,1]*8-4
movies['error'] = (movies['predicted_sentiment']-movies['sentiment']).abs()
movies.error.mean()    #预测结果和真实结果的平均误差
movies['sentiment_ispositive'] = (movies.sentiment>0).astype(int)
movies['predicted_ispositive'] = (movies.predicted_sentiment>0).astype(int)
movies['sentiment predicted_sentiment sentiment_ispositive predicted_ispositive'.split()].head()

MultinomialNB()

1.8698837549746175

,sentiment,predicted_sentiment,sentiment_ispositive,predicted_ispositive
id,,,,
1,2.266667,2.511515,1,1
2,3.533333,3.999904,1,1
3,-0.600000,-3.655976,0,0
4,1.466667,1.940954,1,1
5,1.733333,3.910373,1,1
